In [ ]:
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import glob

def load_and_preprocess_data(file_path):
    """
    Load the dataset and perform initial preprocessing
    """
    df = pd.read_csv(file_path)
    
    # Convert date columns to datetime
    date_columns = ['fields.resolutiondate', 'fields.created', 'fields.updated']
    for col in date_columns:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col])
    
    # Create basic derived features
    if 'fields.created' in df.columns and 'fields.resolutiondate' in df.columns:
        # Only calculate resolution time for resolved issues
        resolved_mask = ~df['fields.resolutiondate'].isna()
        df.loc[resolved_mask, 'resolution_time_hours'] = (
            df.loc[resolved_mask, 'fields.resolutiondate'] - 
            df.loc[resolved_mask, 'fields.created']
        ).dt.total_seconds() / 3600
    
    # Extract day of week and hour of day for temporal analysis
    if 'fields.created' in df.columns:
        df['created_day_of_week'] = df['fields.created'].dt.dayofweek
        df['created_hour'] = df['fields.created'].dt.hour
        df['created_month'] = df['fields.created'].dt.month
        df['created_year'] = df['fields.created'].dt.year
    
    if 'fields.resolutiondate' in df.columns:
        df['resolved_day_of_week'] = df['fields.resolutiondate'].dt.dayofweek
        df['resolved_hour'] = df['fields.resolutiondate'].dt.hour
    
    return df

def aggregate_project_features(df, project_id_col='fields.project.id'):
    """
    Aggregate features at the project level to create a single row per project
    
    Parameters:
    -----------
    df : pandas.DataFrame
        The input DataFrame containing issue-level data
    project_id_col : str
        The column name that contains project IDs
        
    Returns:
    --------
    pandas.DataFrame
        A DataFrame with one row per project and aggregated features
    """
    project_features = {}
    
    # Group data by project
    project_groups = df.groupby(project_id_col)
    
    for project_id, project_df in project_groups:
        features = {}
        
        # Add project identifier
        features['project_id'] = project_id
        features['project_key'] = project_df['fields.project.key'].iloc[0] if 'fields.project.key' in project_df.columns else None
        features['project_name'] = project_df['fields.project.name'].iloc[0] if 'fields.project.name' in project_df.columns else None
        
        # Basic issue counts
        features['total_issues'] = len(project_df)
        
        # 1. Temporal Features
        # Overall project timespan
        if 'fields.created' in project_df.columns and 'fields.resolutiondate' in project_df.columns:
            features['project_start_date'] = project_df['fields.created'].min()
            
            # For end date, use the latest resolved issue or latest update if no resolutions
            resolved_issues = project_df[~project_df['fields.resolutiondate'].isna()]
            if len(resolved_issues) > 0:
                features['project_latest_resolved_date'] = resolved_issues['fields.resolutiondate'].max()
            else:
                features['project_latest_resolved_date'] = None
                
            if 'fields.updated' in project_df.columns:
                features['project_latest_update_date'] = project_df['fields.updated'].max()
            
            # Calculate project duration in days (using latest of either resolution or update)
            if features['project_latest_resolved_date'] is not None:
                latest_date = max(
                    features['project_latest_resolved_date'],
                    features.get('project_latest_update_date', features['project_latest_resolved_date'])
                )
                features['project_duration_days'] = (latest_date - features['project_start_date']).days
            elif 'project_latest_update_date' in features:
                features['project_duration_days'] = (features['project_latest_update_date'] - features['project_start_date']).days
            else:
                features['project_duration_days'] = None
        
        # Resolution time statistics for resolved issues
        resolved_issues = project_df[~project_df['fields.resolutiondate'].isna()]
        if 'resolution_time_hours' in project_df.columns and len(resolved_issues) > 0:
            resolution_times = resolved_issues['resolution_time_hours']
            
            features['avg_resolution_hours'] = resolution_times.mean()
            features['median_resolution_hours'] = resolution_times.median()
            features['min_resolution_hours'] = resolution_times.min()
            features['max_resolution_hours'] = resolution_times.max()
            features['resolution_hours_std'] = resolution_times.std()
            features['total_resolution_hours'] = resolution_times.sum()
            
            # Distribution metrics for resolution times
            if len(resolution_times) > 2:  # Need at least 3 points for skewness/kurtosis
                features['resolution_time_skewness'] = stats.skew(resolution_times)
                features['resolution_time_kurtosis'] = stats.kurtosis(resolution_times)
            
            # Resolution time percentiles
            features['resolution_time_p25'] = resolution_times.quantile(0.25)
            features['resolution_time_p75'] = resolution_times.quantile(0.75)
            features['resolution_time_p90'] = resolution_times.quantile(0.90)
            features['resolution_time_iqr'] = features['resolution_time_p75'] - features['resolution_time_p25']
            
            # Proportion of issues resolved in different time frames
            features['pct_resolved_within_24h'] = (resolution_times <= 24).mean() * 100
            features['pct_resolved_within_week'] = (resolution_times <= 168).mean() * 100  # 168 hours = 1 week
            features['pct_resolved_within_month'] = (resolution_times <= 720).mean() * 100  # 720 hours ≈ 30 days
        
        # Temporal patterns
        if 'fields.created' in project_df.columns:
            # Weekend vs. weekday metrics
            if 'created_day_of_week' in project_df.columns:
                weekend_created = project_df['created_day_of_week'].isin([5, 6])  # 5=Saturday, 6=Sunday
                features['pct_issues_created_on_weekend'] = weekend_created.mean() * 100
            
            if 'resolved_day_of_week' in project_df.columns:
                weekend_resolved = project_df['resolved_day_of_week'].isin([5, 6])
                resolved_issues_count = (~project_df['fields.resolutiondate'].isna()).sum()
                if resolved_issues_count > 0:
                    features['pct_issues_resolved_on_weekend'] = (
                        weekend_resolved & ~project_df['fields.resolutiondate'].isna()
                    ).sum() / resolved_issues_count * 100
            
            # Creation patterns by month
            if 'created_month' in project_df.columns and 'created_year' in project_df.columns:
                # Group issues by year-month and count
                monthly_counts = project_df.groupby(['created_year', 'created_month']).size()
                if len(monthly_counts) > 0:
                    features['max_issues_per_month'] = monthly_counts.max()
                    features['avg_issues_per_month'] = monthly_counts.mean()
                    features['months_with_activity'] = len(monthly_counts)
                    if features['months_with_activity'] > 1:
                        features['issue_creation_volatility'] = monthly_counts.std() / monthly_counts.mean()
        
        # 2. Priority and Issue Type Features
        # Count issues by priority
        if 'priority_name' in project_df.columns:
            priority_counts = project_df['priority_name'].value_counts()
            total_with_priority = priority_counts.sum()
            
            for priority in priority_counts.index:
                col_name = f'priority_{priority.lower().replace(" ", "_")}_count'
                features[col_name] = priority_counts[priority]
                
                # Also add as percentage
                col_pct_name = f'priority_{priority.lower().replace(" ", "_")}_pct'
                features[col_pct_name] = (priority_counts[priority] / total_with_priority * 100) if total_with_priority > 0 else 0
        
        # Alternative approach for priority using binary columns if they exist
        priority_cols = [col for col in project_df.columns if col.startswith('priority_') and col != 'priority_name' and col != 'priority_id']
        if priority_cols:
            for col in priority_cols:
                features[f'{col}_count'] = project_df[col].sum()
                features[f'{col}_pct'] = (project_df[col].sum() / len(project_df) * 100)
        
        # Count issues by type
        if 'issue_type' in project_df.columns:
            type_counts = project_df['issue_type'].value_counts()
            
            for issue_type in type_counts.index:
                col_name = f'type_{issue_type.lower().replace(" ", "_")}_count'
                features[col_name] = type_counts[issue_type]
                
                # Also add as percentage
                col_pct_name = f'type_{issue_type.lower().replace(" ", "_")}_pct'
                features[col_pct_name] = (type_counts[issue_type] / len(project_df) * 100)
        
        # Alternative approach for issue types using binary columns if they exist
        type_cols = [col for col in project_df.columns if col.startswith('type_')]
        if type_cols:
            for col in type_cols:
                features[f'{col}_count'] = project_df[col].sum()
                features[f'{col}_pct'] = (project_df[col].sum() / len(project_df) * 100)
        
        # Priority and issue type combinations
        # For each priority, calculate resolution metrics by issue type
        if 'priority_name' in project_df.columns and 'issue_type' in project_df.columns and 'resolution_time_hours' in project_df.columns:
            for priority in project_df['priority_name'].unique():
                for issue_type in project_df['issue_type'].unique():
                    # Filter issues with this priority and type that have been resolved
                    filtered = project_df[
                        (project_df['priority_name'] == priority) & 
                        (project_df['issue_type'] == issue_type) &
                        ~project_df['fields.resolutiondate'].isna()
                    ]
                    
                    if len(filtered) > 0:
                        prefix = f'priority_{priority.lower().replace(" ", "_")}_type_{issue_type.lower().replace(" ", "_")}'
                        features[f'{prefix}_count'] = len(filtered)
                        features[f'{prefix}_avg_resolution_hours'] = filtered['resolution_time_hours'].mean()
        
        # 3. Issue Dependencies and Complexity
        if 'inward_count' in project_df.columns and 'outward_count' in project_df.columns:
            # Average link counts
            features['avg_inward_links'] = project_df['inward_count'].mean()
            features['avg_outward_links'] = project_df['outward_count'].mean()
            features['avg_total_links'] = project_df['inward_count'].add(project_df['outward_count']).mean()
            
            # Total link counts for the project
            features['total_inward_links'] = project_df['inward_count'].sum()
            features['total_outward_links'] = project_df['outward_count'].sum()
            features['total_links'] = features['total_inward_links'] + features['total_outward_links']
            
            # Issues with many dependencies
            high_dependency_threshold = project_df['inward_count'].quantile(0.75)
            features['pct_issues_with_high_dependencies'] = (
                (project_df['inward_count'] > high_dependency_threshold).mean() * 100
            )
            
            # Link density (average links per issue)
            features['link_density'] = features['total_links'] / features['total_issues'] if features['total_issues'] > 0 else 0
        
        # 4. Resolution Efficiency
        if 'is_resolved' in project_df.columns:
            features['num_resolved_issues'] = project_df['is_resolved'].sum()
            features['pct_resolved_issues'] = features['num_resolved_issues'] / features['total_issues'] * 100 if features['total_issues'] > 0 else 0
        
        # Resolution rate over time
        if 'fields.created' in project_df.columns and 'fields.resolutiondate' in project_df.columns and features.get('project_duration_days', 0) > 0:
            features['resolution_rate_per_day'] = features.get('num_resolved_issues', 0) / features['project_duration_days']
        
        # Resolution efficiency by issue type
        if 'issue_type' in project_df.columns and 'is_resolved' in project_df.columns:
            for issue_type in project_df['issue_type'].unique():
                filtered = project_df[project_df['issue_type'] == issue_type]
                if len(filtered) > 0:
                    type_key = issue_type.lower().replace(" ", "_")
                    features[f'type_{type_key}_resolution_rate'] = filtered['is_resolved'].mean() * 100
        
        # Add features to the project_features dictionary
        project_features[project_id] = features
    
    # Convert to DataFrame
    result_df = pd.DataFrame.from_dict(project_features, orient='index')
    
    # Fill NaN values with appropriate defaults or remove them
    result_df = result_df.fillna({
        'pct_resolved_issues': 0,
        'resolution_rate_per_day': 0,
        # Add other fields as needed
    })
    
    return result_df

def feature_engineering(project_df):
    """
    Perform additional feature engineering on the aggregated project data
    
    Parameters:
    -----------
    project_df : pandas.DataFrame
        DataFrame with one row per project
        
    Returns:
    --------
    pandas.DataFrame
        DataFrame with additional engineered features
    """
    df = project_df.copy()
    
    # Calculate derived metrics
    
    # 1. Efficiency ratio = completed issues / total issues / project duration in weeks
    if 'num_resolved_issues' in df.columns and 'project_duration_days' in df.columns and 'total_issues' in df.columns:
        # Avoid division by zero
        denominator = df['project_duration_days'] * df['total_issues']
        df['weekly_efficiency_ratio'] = np.where(
            denominator > 0,
            df['num_resolved_issues'] * 7 / denominator,
            0  # Default value when denominator is 0
        )
    
    # 2. Complexity-weighted resolution time
    if 'avg_resolution_hours' in df.columns and 'avg_total_links' in df.columns:
        # Add a small constant to ensure we don't multiply by zero
        df['complexity_weighted_resolution_time'] = df['avg_resolution_hours'] * (df['avg_total_links'] + 1)
    
    # 3. Priority balance - ratio of high priority to low priority issues
    high_priority_cols = [col for col in df.columns if ('priority_critical' in col or 'priority_blocker' in col or 'priority_major' in col) and '_count' in col]
    low_priority_cols = [col for col in df.columns if ('priority_minor' in col or 'priority_trivial' in col) and '_count' in col]
    
    if high_priority_cols and low_priority_cols:
        high_priority_sum = df[high_priority_cols].sum(axis=1)
        low_priority_sum = df[low_priority_cols].sum(axis=1)
        
        # Avoid division by zero
        df['high_to_low_priority_ratio'] = np.where(
            low_priority_sum > 0,
            high_priority_sum / low_priority_sum,
            high_priority_sum  # If no low priority issues, just use the high priority count
        )
    
    # 4. Bug ratio - proportion of bugs to total issues
    bug_cols = [col for col in df.columns if 'type_bug' in col and '_count' in col]
    if bug_cols and 'total_issues' in df.columns:
        df['bug_ratio'] = df[bug_cols].sum(axis=1) / df['total_issues']
    
    # 5. Creation-resolution balance - how evenly distributed is the workload
    if 'avg_issues_per_month' in df.columns and 'resolution_rate_per_day' in df.columns:
        monthly_creation_rate = df['avg_issues_per_month'] / 30  # Convert to daily rate
        # Balance = 1 means perfect balance, < 1 means resolution is slower than creation
        df['creation_resolution_balance'] = np.where(
            monthly_creation_rate > 0,
            df['resolution_rate_per_day'] / monthly_creation_rate,
            0  # Default value when monthly_creation_rate is 0
        )
    
    # 6. Weighted priority score
    priority_weight_cols = {
        'priority_blocker': 5, 
        'priority_critical': 4, 
        'priority_major': 3, 
        'priority_minor': 2, 
        'priority_trivial': 1
    }
    
    priority_count_cols = [col for col in df.columns if any(p in col for p in priority_weight_cols.keys()) and '_count' in col]
    
    if priority_count_cols and 'total_issues' in df.columns:
        weighted_sum = 0
        for col in priority_count_cols:
            # Extract the priority name from the column name
            for priority_name, weight in priority_weight_cols.items():
                if priority_name in col:
                    weighted_sum += df[col] * weight
                    break
        
        df['weighted_priority_score'] = weighted_sum / df['total_issues']
    
    # 7. Issue diversity - entropy of issue type distribution
    type_pct_cols = [col for col in df.columns if col.startswith('type_') and '_pct' in col]
    if type_pct_cols:
        # Convert percentages to proportions
        proportions = df[type_pct_cols].div(100)
        
        # Calculate entropy for each row (project)
        def entropy(row):
            # Filter out zero values to avoid log(0)
            props = row[row > 0]
            if len(props) == 0:
                return 0
            return -sum(props * np.log2(props))
        
        df['issue_type_entropy'] = proportions.apply(entropy, axis=1)
    
    # 8. Project velocity over time (if there's enough temporal data)
    if 'num_resolved_issues' in df.columns and 'months_with_activity' in df.columns and df['months_with_activity'].max() > 1:
        df['monthly_velocity'] = df['num_resolved_issues'] / df['months_with_activity']
    
    return df

def process_repository_folders(input_base_dir, output_base_dir):
    """
    Process all repositories in the input base directory and save the results to the output base directory.
    
    Parameters:
    -----------
    input_base_dir : str
        Base directory containing repository folders with issue data
    output_base_dir : str
        Base directory to save processed project-level data
    """
    # Create output directory if it doesn't exist
    os.makedirs(output_base_dir, exist_ok=True)
    
    # Get a list of all repository folders
    repo_folders = [f for f in os.listdir(input_base_dir) if os.path.isdir(os.path.join(input_base_dir, f))]
    
    print(f"Found {len(repo_folders)} repository folders in {input_base_dir}")
    
    # Process each repository folder
    for repo_folder in repo_folders:
        repo_path = os.path.join(input_base_dir, repo_folder)
        repo_output_dir = os.path.join(output_base_dir, repo_folder)
        
        # Create output repository folder if it doesn't exist
        os.makedirs(repo_output_dir, exist_ok=True)
        
        # Find all CSV files in the repository folder
        csv_files = glob.glob(os.path.join(repo_path, "*.csv"))
        
        if not csv_files:
            print(f"No CSV files found in {repo_path}, skipping...")
            continue
        
        print(f"Processing repository: {repo_folder} - Found {len(csv_files)} CSV files")
        
        # Process each CSV file
        for csv_file in csv_files:
            file_name = os.path.basename(csv_file)
            print(f"  Processing file: {file_name}")
            
            try:
                # Load and preprocess the data
                df = load_and_preprocess_data(csv_file)
                
                # Aggregate features by project
                project_df = aggregate_project_features(df)
                
                # Perform feature engineering
                final_df = feature_engineering(project_df)
                
                # Save to output file
                output_file = os.path.join(repo_output_dir, f"project_level_{file_name}")
                final_df.to_csv(output_file, index=False)
                
                print(f"  Saved project-level features to {output_file}")
            except Exception as e:
                print(f"  Error processing {file_name}: {str(e)}")

# Paths specific to the notebook environment
INPUT_BASE_DIR = "/Users/diegodias/Documents/Projects/JiraDataset/FeatureCleaning/jira_extracted_data"
OUTPUT_BASE_DIR = "/Users/diegodias/Documents/Projects/JiraDataset/OverallProjectEstimation/project_level_data"

# If this script is run directly, execute the pipeline with the specified input and output paths
if __name__ == "__main__":
    print(f"Starting pipeline to process repository folders...")
    process_repository_folders(INPUT_BASE_DIR, OUTPUT_BASE_DIR)
    print("Pipeline completed successfully!")

In [ ]:
import pandas as pd
import dtale

# Replace this path with the location of your CSV file
csv_file_path = "./project_level_data/MongoDB/project_level_10000_Core_Server.csv"

# Read the CSV into a DataFrame
df = pd.read_csv(csv_file_path)

# Start a D-Tale session and open it in the browser
d = dtale.show(df, ignore_duplicate=True, allow_cell_edits=False)
d.open_browser()